### torch.autograd 를 사용한 지동 미분
- 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파입니다.<br>
이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정됩니다.
- 이러한 변화도를 계산하기 위해 `PyTorch` 에는 `torch.autograd` 라고 불리는 자동 미분 엔진이 내장되어 있습니다.<br>
이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원합니다.
- 입력 `x`, 매개변수 `w`와 `b` , 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 가정하겠습니다.<br>
`PyTorch` 에서는 다음과 같이 정의할 수 있습니다.

In [166]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(f"x: {x}")
print(f"y: {y}")
print(f"Weight: {w}")
print(f"Bias: {b}")
print(f"z: {z}")
print(f"Loss: {loss}")

x: tensor([1., 1., 1., 1., 1.])
y: tensor([0., 0., 0.])
Weight: tensor([[-0.2656, -0.6922, -1.1533],
        [-2.0894, -0.7543, -0.1897],
        [ 0.8913, -0.6837,  0.8136],
        [ 0.4971,  1.1328, -1.9456],
        [-0.0440, -0.7898, -0.1048]], requires_grad=True)
Bias: tensor([ 0.0741, -0.1028,  0.1825], requires_grad=True)
z: tensor([-0.9365, -1.8899, -2.3973], grad_fn=<AddBackward0>)
Loss: 0.18617337942123413


#### Tensor, Function 과 연산 그래프(Computational graph)
- 신경망에서 `w`, `b` 는 최적화를 해야 하는 매개변수(parameter)이다.<br>
따라서 이러한 변수들에 대한 손실 함수(비용 함수)의 변화도를 계산할 수 있어야 한다.<br>
이를 위해서 해당 텐서에 `requires_grad` 속성을 설정한다.
    - NOTE: `requires_grad` 의 값은 텐서를 생성할 때 설정하거나,<br>
    나중에 `x.requires_grad_(True)` 메소드를 사용하여 설정할 수도 있다.

In [167]:
# 연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 Function 클래스의 객체이다.
# 이 객체는 '순전파'방향으로 함수를 계산하는 방법과, '역전파' 단계에서 도함수(derivative)를 계산하는 방법이 있다.
# 역전파 함수에 대한 reference 는 텐서의 'grad_fn' 속성에 저장된다.
print(z.grad_fn)
print(loss.grad_fn)

#### 변화도(Gradient) 계산하기


- 신경망에서 매개변수와 가중치(기울기)를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야 한다.<br>
즉, `x`와 `y`의 일부 고정값에서 $\frac{\partial loss}{\partial w}$ 와 $\frac{\partial loss}{\partial b}$ 가 필요하다.<br>
이러한 도함수를 계산하기 위해, `loss.backward()` 를 호출한 다음 `w.grad`와 `b.grad`에서 값을 가져온다.
    - NOTE: 연산 그래프의 잎 노드(leaf, 단말 노드)들 중 `requires_grad` 속성이 `True` 로 설정된<br>
    노드들의 `grad` 속성만 구할 수 있다. 그래프의 다른 모든 노드에서는 변화도가 유효하지 않다.<br>
    성능 상의 이유로, 현재 실습하는 그래프에서의 `backward` 를 사용한 변화도 계산은 한 번만 수행 가능하다.<br>
    만약 동일한 그래프에서 여러번의 `backward` 호출이 필요하다면, `backward` 호출 시에<br>
    `retain_graph=True`를 전달해야 한다.

In [173]:
loss.backward(retain_graph=True)
print(w.grad)
print(b.grad)

tensor([[0.5632, 0.2625, 0.1668],
        [0.5632, 0.2625, 0.1668],
        [0.5632, 0.2625, 0.1668],
        [0.5632, 0.2625, 0.1668],
        [0.5632, 0.2625, 0.1668]])
tensor([0.5632, 0.2625, 0.1668])


#### 변화도 추적 멈추기
- 기본적으로, `requires_grad=True` 인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원한다.<br>
그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산만 필요한 경우에는,<br>
이러한 추적이나 지원이 필요 없을 수 있다. 연산코드를 `torch.no_grad()` 블록으로 둘러싸서 연산 추적을<br>
멈출 수 있다.
- 변화도 추적을 멈춰야 하는 경우와 그에 따른 이유는 다음과 같다.
    - 사전 학습(pre-trained)된 신경망을 미세조정(fine tuning) 할 때 신경망의 일부 매개변수를<br>
    고정된 매개변수(frozen parameter)로 표시하는 경우
    - 변화도를 추적하지 않는 텐서의 연산이 더 효율적이어서 순전파 단계만 수행할 때 연산 속도가 향상되는 경우

In [174]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


In [175]:
# 위와 동일한 결과를 얻는 방법은 텐서에 'detach()' 메소드를 사용하는 것이다.
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False
